## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-17-18-49-24 +0000,nyt,Live Updates: Trump Equivocates on Ukraine’s R...,https://www.nytimes.com/live/2025/10/17/us/tru...
1,2025-10-17-18-49-04 +0000,nypost,LI man charged with killing teen outside NYC b...,https://nypost.com/2025/10/17/us-news/li-man-c...
2,2025-10-17-18-48-32 +0000,nypost,Near cat-astrophe: Small-town cat clings to va...,https://nypost.com/2025/10/17/us-news/small-to...
3,2025-10-17-18-46-03 +0000,nyt,Maps: Where the U.S. Is Building Up Military F...,https://www.nytimes.com/2025/10/17/us/politics...
4,2025-10-17-18-46-00 +0000,wsj,Proxy Adviser ISS Urges Tesla Holders to Rejec...,https://www.wsj.com/business/autos/proxy-advis...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2322/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,61
92,new,16
289,china,15
37,military,13
105,against,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
338,2025-10-16-20-59-00 +0000,wsj,The four-star U.S. Navy admiral in charge of U...,https://www.wsj.com/politics/national-security...,141
343,2025-10-16-20-34-59 +0000,nypost,Ex-Trump national security adviser John Bolton...,https://nypost.com/2025/10/16/us-news/ex-trump...,121
301,2025-10-16-23-01-00 +0000,wsj,Trump Says He Will Meet With Putin in Budapest...,https://www.wsj.com/world/russia/trump-and-put...,116
316,2025-10-16-21-54-57 +0000,cbc,"John Bolton, Trump's ex-national security advi...",https://www.cbc.ca/news/world/john-bolton-char...,115
169,2025-10-17-11-39-43 +0000,bbc,Zelensky to make case for missiles at White Ho...,https://www.bbc.com/news/articles/crmxz37nv3zo...,110


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
338,141,2025-10-16-20-59-00 +0000,wsj,The four-star U.S. Navy admiral in charge of U...,https://www.wsj.com/politics/national-security...
247,74,2025-10-17-03-46-08 +0000,nypost,"Zohran Mamdani dodges attacks, straight answer...",https://nypost.com/2025/10/16/us-news/mamdani-...
67,60,2025-10-17-16-42-19 +0000,nypost,John Bolton spotted still using White House ba...,https://nypost.com/2025/10/17/us-news/john-bol...
219,45,2025-10-17-09-00-54 +0000,wapo,"Missing 1,900-year-old Roman artifact was foun...",https://www.washingtonpost.com/nation/2025/10/...
225,41,2025-10-17-08-10-07 +0000,wsj,Stock Market Today: Dow Rises; Trump Says Extr...,https://www.wsj.com/livecoverage/stock-market-...
152,39,2025-10-17-13-05-55 +0000,wapo,"Trump plans Putin summit on Ukraine, raising s...",https://www.washingtonpost.com/politics/2025/1...
110,38,2025-10-17-15-18-45 +0000,nyt,"Baek Sehee, South Korean Author of ‘I Want to ...",https://www.nytimes.com/2025/10/17/arts/baek-s...
308,37,2025-10-16-22-19-45 +0000,nypost,Shocking video captures Tren de Aragua gang le...,https://nypost.com/2025/10/16/world-news/fugit...
18,36,2025-10-17-18-19-43 +0000,nypost,Jeffrey Epstein-linked Prince Andrew announces...,https://nypost.com/2025/10/17/world-news/princ...
52,36,2025-10-17-17-06-41 +0000,cbc,Federal government to push ahead with plan to ...,https://www.cbc.ca/news/politics/politics-bord...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
